<a href="https://colab.research.google.com/github/shruti-singh/PaperAcceptancePrediction/blob/master/get_data_ICLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install --user openreview-py

In [0]:
import pandas as pd
import openreview
from pprint import pprint
import pickle

In [0]:
uname = "singh_shruti@iitgn.ac.in"
passwd = ""

In [0]:
client = openreview.Client(baseurl='https://openreview.net', username=uname, password=passwd)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_dir = "/content/drive/My Drive/Coursework/Sem2/ML/PaperAcceptancePrediction/"

In [0]:
iclr2020_group = client.get_group("ICLR.cc/2020/Conference")

In [0]:
blind_sub_iter = openreview.tools.iterget_notes(client, invitation="ICLR.cc/2020/Conference/-/Blind_Submission")
papers_meta = []

for blindsub in blind_sub_iter:
	papers_meta.append(blindsub.__dict__)

df = pd.DataFrame(papers_meta)

In [0]:
pprint(papers_meta[0])
print(df.columns)

{'cdate': 1569438892732,
 'content': {'_bibtex': '@inproceedings{\n'
                        'Shu2020Understanding,\n'
                        'title={Understanding Architectures Learnt by '
                        'Cell-based Neural Architecture Search},\n'
                        'author={Yao Shu and Wei Wang and Shaofeng Cai},\n'
                        'booktitle={International Conference on Learning '
                        'Representations},\n'
                        'year={2020},\n'
                        'url={https://openreview.net/forum?id=BJxH22EKPS}\n'
                        '}',
             'abstract': 'Neural architecture search (NAS) searches '
                         'architectures automatically for given tasks, e.g., '
                         'image classification and language modeling. '
                         'Improving the search efficiency and effectiveness '
                         'has attracted increasing attention in recent years. '
                  

In [0]:
papers_dict = {}

for paper in papers_meta:
    forum_id = paper["forum"]
    papers_dict[forum_id] = paper

In [0]:
paper_decision_dict = {}

decision_iterator = openreview.tools.iterget_notes(client, invitation="ICLR.cc/2020/Conference/Paper.*/-/Decision")

for review in decision_iterator:
    if review.forum in paper_decision_dict:
        paper_decision_dict[review.forum]["decision_1"] = review.__dict__
    else:
        paper_decision_dict[review.forum] = {}
        paper_decision_dict[review.forum]["decision"] = review.__dict__

In [0]:
print(list(paper_decision_dict.keys())[0:3])
paper_decision_dict['BklSv34KvB']

['Syx4wnEtvH', 'BklSv34KvB', 'BklSwn4tDH']


{'decision': {'cdate': 1576798684487,
  'content': {'comment': 'The authors propose a new mini-batch selection method for training deep NNs. Rather than random sampling, selection is based on a sliding window of past model predictions for each sample and uncertainty about those samples. Results are presented on MNIST and CIFAR.\n\nThe reviewers agreed that this is an interesting idea which was clearly presented, but had concerns about the strength of the experimental results, which showed only a modest benefit on relatively simple datasets. In the rebuttal period, the authors added an ablation study and additional results on Tiny-ImageNet. However, the results on the new dataset seem very marginal, and R1 did not feel that all of their concerns were addressed. I’m inclined to agree that more work is required to prove the generalizability of this approach before it’s suitable for acceptance.\n',
   'decision': 'Reject',
   'title': 'Paper Decision'},
  'ddate': None,
  'details': {'repl

In [0]:
# # Ensure if paper decision is not more than one each paper.
# for k, v in paper_decision_dict.items():
#     if len(v) > 1:
#         print(k)

In [0]:
off_rev_dict = {}

official_review_iter = openreview.tools.iterget_notes(client, invitation='ICLR.cc/2020/Conference/Paper.*/-/Official_Review')

# Note in review, forum is the paperid, and id is the id of the review. For paper, both these fields are same. For eg, link to a review is of form:
# https://openreview.net/forum?id=r1x0lxrFPS&noteId=HJgd6tglcS ,  where id=r1x0lxrFPS is forum/paper_id and noteId=HJgd6tglcS is the review_id(everything is a note).
for off_rev in official_review_iter:
    if off_rev.forum in off_rev_dict:
        off_rev_dict[off_rev.forum].append(off_rev.__dict__)
    else:
        off_rev_dict[off_rev.forum] = [off_rev.__dict__]

In [0]:
print(len(papers_dict))
print(len(paper_decision_dict))
print(len(off_rev_dict))

k1 = set(papers_dict.keys())
k2 = set(paper_decision_dict.keys())
k3 = set(off_rev_dict.keys())

print("\n\nk1:%s, k2:%s, k3:%s \n"%(len(k1), len(k2), len(k3)))
# Ensures that we have the corresponding paper decisions for all papers in the daaset
print(k1-k2)
# These are DeskRejected papers or withdrawn papers
print(k3-k1)
# All papers in the dataset have the official reviews
print(k1-k3)

2213
2213
2561


k1:2213, k2:2213, k3:2561 

set()
{'SkeJMCVFDS', 'HkebMlrFPS', 'S1ebsJrFwS', 'SklEs2EYvS', 'Skx1dhNYPS', 'HJgLlgBKvH', 'ryegXAVKDB', 'Bke5aJBKvH', 'H1lkCTVFwB', 'B1gL904FwH', 'S1xRbxHYDr', 'BJluPerYvB', 'r1g4onVYvS', 'BJxRrlBFwB', 'BJlo91BYPr', 'rkxsgkHKvH', 'HJg0_eBFwB', 'B1lf4yBYPr', 'H1xTEgBKvB', 'BJl_a2VYPH', 'BJxD11HFDS', 'BJg_FgBtPH', 'SkgBfaNKPr', 'B1lpxCVYDr', 'HygUBaEFDB', 'Skg7TerKPH', 'rJlaY3VYDr', 'rJlHea4Kvr', 'SylDPJrYvS', 'BJxoz1rKwr', 'SJenMR4twH', 'SyxytxBFDr', 'H1l0e6VKDS', 'rkgZaT4tDr', 'SJgubJrKPr', 'BJxg7eHYvB', 'B1gtK0NKwr', 'HklJ4gSFPS', 'HJgjuCVKwS', 'BJgMn0EYPB', 'SJxngREtDB', 'Sklia3EFPH', 'SkgC6yHtvB', 'r1xQH0EtvH', 'r1lI3ertwH', 'rJgKzlSKPH', 'H1xtU6VtwH', 'ryl260NYDr', 'HJgJNCEKPr', 'HklOjkHKDr', 'SyxoygBKwB', 'SJlzvTNtvS', 'Hklk6xrYPB', 'Hyljn1SFwr', 'H1lnZlHYDS', 'B1xgWpEKvH', 'H1l7AkrFPS', 'r1xBoxBYDH', 'BJgpDyHKwH', 'SJxivAEYvr', 'r1gmu1SYDS', 'SJxbu6VKDr', 'B1gi-TVKwB', 'S1gV36NKPS', 'BkxnKkrtvS', 'rJg4GgHKPB', 'Bkx5ceHFwH', 'SJgJ-AEYP

In [0]:
avg_reviews = 0
for review in off_rev_dict:
    avg_reviews += len(off_rev_dict[review])
print("Average number of reviews for papers: ", avg_reviews/len(off_rev_dict))

Average number of reviews for papers:  3.0359234673955484


**Save pickle files**

In [0]:
with open(data_dir + "papers_2020.pkl", "wb") as f:
    pickle.dump(papers_dict, f)

with open(data_dir + "paper_decision_dict_2020.pkl", "wb") as f:
    pickle.dump(paper_decision_dict, f)

with open(data_dir + "off_rev_dict_2020.pkl", "wb") as f:
    pickle.dump(off_rev_dict, f)

**GET 2019 DATA**

In [0]:
iclr2019_group = client.get_group("ICLR.cc/2019/Conference")
print(iclr2019_group)

{'cdate': None,
 'ddate': None,
 'details': {'writable': False},
 'id': 'ICLR.cc/2019/Conference',
 'members': ['iclr2019admin@openreview.net'],
 'nonreaders': [],
 'readers': ['everyone'],
 'signatories': ['ICLR.cc/2019/Conference'],
 'signatures': ['~Super_User1'],
 'web': '// ------------------------------------\n'
        '// Advanced venue homepage template\n'
        '//\n'
        '// This webfield displays the conference header (#header), the submit '
        'button (#invitation),\n'
        '// and a tabbed interface for viewing various types of notes.\n'
        '// ------------------------------------\n'
        '\n'
        '// Constants\n'
        "var CONFERENCE = 'ICLR.cc/2019/Conference';\n"
        "var INVITATION = CONFERENCE + '/-/Submission';\n"
        "var BLIND_INVITATION = CONFERENCE + '/-/Blind_Submission';\n"
        "var WITHDRAWN_INVITATION = CONFERENCE + '/-/Withdrawn_Submission';\n"
        '\n'
        'var initialPageLoad = true;\n'
        '\n'
       

In [0]:
blind_sub_iter_2019 = openreview.tools.iterget_notes(client, invitation="ICLR.cc/2019/Conference/-/Blind_Submission")
papers_meta_dict = {}

for blindsub in blind_sub_iter_2019:
    fid = blindsub.forum
    if fid in papers_meta_dict:
	    print("Duplicate entry: ", fid)
    else:
        papers_meta_dict[fid] = blindsub.__dict__

# df = pd.DataFrame(papers_meta)

In [0]:
withdrawn_iter = openreview.tools.iterget_notes(client, invitation="ICLR.cc/2019/Conference/-/Withdrawn_Submission")

withdrawn_list = []

for w in withdrawn_iter:
    withdrawn_list.append(w)

In [0]:
print("Papers meta: ", len(papers_meta_dict))
print("Withdrawn papers: ", len(withdrawn_list))

# Check all the papers are retrieved
assert (len(withdrawn_list)+len(papers_meta_dict)) == 1579

Papers meta:  1419
Withdrawn papers:  160


In [0]:
paper_decision_dict = {}

# Paper.*/Meta_Review
decision_iterator = openreview.tools.iterget_notes(client, invitation="ICLR.cc/2019/Conference/-/Paper.*/Meta_Review")

for review in decision_iterator:
    # print(review)
    # break
    if review.forum in paper_decision_dict:
        print("Encountered multiple for forum: ", review.forum)
        # paper_decision_dict[review.forum]["decision_1"] = review.__dict__
    else:
        paper_decision_dict[review.forum] = {}
        paper_decision_dict[review.forum]["decision"] = review.__dict__

In [0]:
len(paper_decision_dict)

1419

In [0]:
# Official_Review
off_rev_dict = {}

official_review_iter = openreview.tools.iterget_notes(client, invitation='ICLR.cc/2019/Conference/-/Paper.*/Official_Review')

# Note in review, forum is the paperid, and id is the id of the review. For paper, both these fields are same. For eg, link to a review is of form:
# https://openreview.net/forum?id=r1x0lxrFPS&noteId=HJgd6tglcS ,  where id=r1x0lxrFPS is forum/paper_id and noteId=HJgd6tglcS is the review_id(everything is a note).
for off_rev in official_review_iter:
    if off_rev.forum in off_rev_dict:
        off_rev_dict[off_rev.forum].append(off_rev.__dict__)
    else:
        off_rev_dict[off_rev.forum] = [off_rev.__dict__]

In [0]:
len(off_rev_dict)

1565

In [0]:
print(len(papers_meta_dict))
print(len(paper_decision_dict))
print(len(off_rev_dict))

k1 = set(papers_meta_dict.keys())
k2 = set(paper_decision_dict.keys())
k3 = set(off_rev_dict.keys())

print("\n\nk1:%s, k2:%s, k3:%s \n"%(len(k1), len(k2), len(k3)))
# Ensures that we have the corresponding paper decisions for all papers in the daaset
print(k1-k2)
# These are DeskRejected papers or withdrawn papers
print(k3-k1)
# All papers in the dataset have the official reviews
print(k1-k3)

1419
1419
1565


k1:1419, k2:1419, k3:1565 

set()
{'HkNN7nR5Ym', 'B1GRtj05t7', 'HJz1vo0cYX', 'rylU8oRctX', 'B1gTE2AcKQ', 'ByWMz305FQ', 'rJxA-h05KQ', 'HkxzDiAcK7', 'rkzcvoA9YX', 'rJEGwo0cFX', 'rJe-LiA5YX', 'ryxfHnCctX', 'H1GWAoRcKX', 'S1fNJhRqFX', 'SyxdbnR9YQ', 'BJgFcj0qKX', 'rkl85oRqYX', 'rklvnjRqY7', 'ryxMjoRcYm', 'BJlhEs09YQ', 'H1eRIoA5Y7', 'r1lFYoRcFm', 'rkGqLoR5tX', 'BJf9k305Fm', 'H1xLsjAqtX', 'r1eWW2RqFX', 'HJN6DiAcKQ', 'BJxz5jRcFm', 'B1enCo0cK7', 'rygPUoR9YQ', 'Sye8S209KX', 'BJgAfh09tm', 'BJxfJnC9YX', 'rJlfzhA9Y7', 'rJlGdsC9Ym', 'r1lcM3AcKm', 'Syfe2iR5FQ', 'Sylw7nCqFQ', 'SkgYciAqF7', 'BkeSusCcYm', 'HyxH2o05FQ', 'ryEquiR9KX', 'BJlr0j0ctX', 'SygeznA9YX', 'H1eZ6sRcFm', 'rkMlSnAqYX', 'S1eKJ3R5KQ', 'BJgWl3A5YX', 'r1MVuoCctX', 'rylBZ305KQ', 'SkGMOi05FQ', 'r1e-nj05FQ', 'r1ejxnCctX', 'HkzL4hR9Ym', 'S1g6zn09tm', 'HJM4SjR5KQ', 'SkfQAiA9YX', 'ryxnDoRcK7', 'r1GaAjRcF7', 'B1x0E2C5tQ', 'SJzwb2RcK7', 'rJegl2C9K7', 'ryxtE3C5Fm', 'SygNooCqY7', 'r1g5Gh05KQ', 'rkelDoCqFX', 'B1GHb2RqYX', 'BJlif3C5F

**Save pickle files**

In [0]:
with open(data_dir + "papers_2019.pkl", "wb") as f:
    pickle.dump(papers_meta_dict, f)

with open(data_dir + "paper_decision_dict_2019.pkl", "wb") as f:
    pickle.dump(paper_decision_dict, f)

with open(data_dir + "off_rev_dict_2019.pkl", "wb") as f:
    pickle.dump(off_rev_dict, f)